In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "ThePwo/",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
    token = "hf_"
)

In [ ]:
import random

def get_last_message(output):
    parts = output.rsplit('\n\n', 1)
    if len(parts) > 1:
        return parts[-1].strip().replace('<|eot_id|>', '')
    return None

import textwrap

def print_wrapped(text):
    wrapped_text = textwrap.fill(text, width=80)
    print(wrapped_text)

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"},
)

FastLanguageModel.for_inference(model)

In [ ]:
messages = []

for step in range(50):
    if step == 0 or random.random() < 0.5:
        messages.append({"from": "human", "value": input(">> User: ")})

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True,
        return_tensors = "pt",
    ).to("cuda")

    outputs = model.generate(input_ids = inputs, max_new_tokens = 512, use_cache=True, temperature=1.0)
    response = get_last_message(tokenizer.batch_decode(outputs)[0])
    messages.append({"from": "gpt", "value": response})
    messages = messages[-10:]
    print_wrapped(response)